In [2]:
import tensorflow as tf
from time import time

In [11]:
# timer utility
def timer(start: float, end: float) -> str:
        """
        Timer function. Compute execution time from strart to end (end - start).
        :param start: start time
        :param end: end time
        :return: end - start
        """
        hours, rem = divmod(end - start, 3600)
        minutes, seconds = divmod(rem, 60)
        return "{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds)

In [5]:
# params
num_words = 50000
embedding_size = 50
maxlen = 250

In [6]:
# load the dataset
train_data, test_data = tf.keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=num_words,
    skip_top=0,
    maxlen=maxlen,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3,
)

In [7]:
x_train, y_train = train_data
x_test, y_test = test_data

In [8]:
# quickly pad the datasets
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

Define the model

In [12]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Embedding(num_words, embedding_size, input_length=maxlen),
        tf.keras.layers.LSTM(128),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 50)           2500000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,591,777
Trainable params: 2,591,777
Non-trainable params: 0
_________________________________________________________________


Train

In [10]:
start = time()
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)
end = time()
print("Training time:", timer(start, end))

Epoch 1/3
268/268 [==============================] - 18s 66ms/step - loss: 0.6399 - accuracy: 0.6279 - val_loss: 0.3687 - val_accuracy: 0.8447
Epoch 2/3
268/268 [==============================] - 18s 66ms/step - loss: 0.2601 - accuracy: 0.8997 - val_loss: 0.3216 - val_accuracy: 0.8626
Epoch 3/3
268/268 [==============================] - 18s 67ms/step - loss: 0.1329 - accuracy: 0.9553 - val_loss: 0.3671 - val_accuracy: 0.8403


TypeError: timer() missing 1 required positional argument: 'end'

Evaluation just for fun

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))